# Bernstein Conference 2023: Triad counts

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

In [2]:
# figs_path = None
figs_path = './bernstein2023_figs'
if not os.path.exists(figs_path):
    os.makedirs(figs_path)

### Triad counts

In [3]:
adj_files = ['/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/structural_comparator_v2/SSCx-HexO1-Release/data/Orig_Adjacency_Hex0EE.pickle',
             '/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/structural_comparator_v2/SSCx-HexO1-Release/data/Order-1_Adjacency_Hex0EE.pickle',
             '/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/structural_comparator_v2/SSCx-HexO1-Release/data/Order-2_Adjacency_Hex0EE.pickle',
             '/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/structural_comparator_v2/SSCx-HexO1-Release/data/Order-3_Adjacency_Hex0EE.pickle',
             '/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/structural_comparator_v2/SSCx-HexO1-Release/data/Order-4_Adjacency_Hex0EE.pickle',
             '/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/structural_comparator_v2/SSCx-HexO1-Release/data/Order-5_Adjacency_Hex0EE.pickle']
plot_names = ['Original',
              '1st order',
              '2nd order',
              '3rd order',
              '4th order',
              '5th order']

In [59]:
# Load adjacency matrices (optional: subsampling)
sub_fraction = 0.1
sub_seed = 0

adj_mats = []
for fn in adj_files:
    with open(fn, 'rb') as f:
        adj_dict = pickle.load(f)
    _adj = adj_dict['adj']['data']

    # Subsample matrices
    np.random.seed(sub_seed)
    sel_idx = np.random.choice(2, _adj.shape[0], p=[1.0 - sub_fraction, sub_fraction]).astype(bool)
    _adj = _adj[:, sel_idx][sel_idx, :]

    adj_mats.append(_adj)
    print(_adj.__repr__())


<2711x2711 sparse matrix of type '<class 'numpy.bool_'>'
	with 72757 stored elements in Compressed Sparse Row format>
<2711x2711 sparse matrix of type '<class 'numpy.bool_'>'
	with 73734 stored elements in Compressed Sparse Row format>
<2711x2711 sparse matrix of type '<class 'numpy.bool_'>'
	with 73796 stored elements in Compressed Sparse Row format>
<2711x2711 sparse matrix of type '<class 'numpy.bool_'>'
	with 73201 stored elements in Compressed Sparse Row format>
<2711x2711 sparse matrix of type '<class 'numpy.bool_'>'
	with 72549 stored elements in Compressed Sparse Row format>
<2711x2711 sparse matrix of type '<class 'numpy.bool_'>'
	with 71295 stored elements in Compressed Sparse Row format>


In [60]:
import importlib
spec = importlib.util.spec_from_file_location('topology', '/gpfs/bbp.cscs.ch/home/pokorny/git/connectome_analysis/src/connalysis/network/topology.py')
topology = importlib.util.module_from_spec(spec)
spec.loader.exec_module(topology)

In [62]:
for _adj, _nm in zip(adj_mats, plot_names):
    print(f'Running "{_nm}" triad counting...')
    if sub_fraction < 1.0:
        fn = os.path.join(figs_path, f'triad_counts__sub{sub_fraction}__{_nm}.txt')
    else:
        fn = os.path.join(figs_path, f'triad_counts__{_nm}.txt')
    if os.path.exists(fn):
        print(f'>>> "{_nm}" counts already exist ... SKIPPING')
        continue
    triad_counts = topology.count_triads_fully_connected(_adj)
    print(f'>>> "{_nm}" triad counts: {triad_counts}\n')
    np.savetxt(fn, triad_counts, fmt='%d')

Running "Original" triad counting...
Testing 1984608 potential triadic pairs
Time spent finding triads: 25.55913472175598
Found 4332298 connected triads
Time spent classifying triads: 236.72150874137878
>>> "Original" triad counts: [1710641 1171871 1251664   68581   61872   54894    7139     944    1567
    1550    1456     117       2]

Running "1st order" triad counting...
Testing 2427632 potential triadic pairs
Time spent finding triads: 26.99901819229126
Found 3911991 connected triads
Time spent classifying triads: 234.37580490112305
>>> "1st order" triad counts: [1925483  959379  963284   19631   18684   18639    6445      88     187
      83      87       1       0]

Running "2nd order" triad counting...
Testing 2253360 potential triadic pairs
Time spent finding triads: 25.280606269836426
Found 3924623 connected triads
Time spent classifying triads: 207.68455696105957
>>> "2nd order" triad counts: [1895996  944423  947652   28565   48049   47838    9532     645     929
     460  

In [ ]:
# Trim PNGs
# mogrify -trim *.png